In [1]:
# Same code as arch hall, copy paste training and model code
# Save ANN, LSTM CL and BL models
# Save the models in seperate folders/database

In [2]:
import pytz
from datetime import datetime
import pandas as pd
import numpy as np
import os
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import keras
import pandas as pd
from itables import show
from keras.models import Sequential # type: ignore
from keras.layers import Dense, Input, LSTM, Embedding, SimpleRNN, GRU, Bidirectional, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError       
from keras.regularizers import l2
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error
import scipy
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sktime.transformations.series.holiday import HolidayFeatures
from holidays import country_holidays, financial_holidays
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from padasip.filters import FilterRLS
import pickle
from sklearn.model_selection import TimeSeriesSplit
np.printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from weather_utils import WeatherUtils
import seaborn as sns
import holidays
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


In [3]:
saved_ann_models_path = './../Saved_Models/ANN_Saved_Models'
saved_lstm_models_path = './../Saved_Models/LSTM_Saved_Models'
saved_rls_combiner_models_path = './../Saved_Models/RLS_Combiner_Saved_Models'
feeders_metadata_path = './../Data/Filtered_Feeders_Metadata/Final_Selected_Feeders_Data_with_Coordinates.csv'
feeders_root_path = "./../Data/Feeder_Data/"
openmeteo_weather_data_path = "./../Data/Feeder_Weather_Combined_Data/"
filtered_feeders_metadata_file_path = "../Data/Filtered_Feeders_Metadata/Final_Selected_Feeders_Data_with_Coordinates.csv"
weather_data_path = "./../Data/Weather_Data/high_resolution_weather_arch_hall.csv"
saved_models_path = "./../Saved_Models"
saved_results_path = "./../Data/Results"
train_stats_path = './../Data/Filtered_Feeders_Metadata/Train_Stats'

end_train = "2024-06-16 23:59:59"
end_val = "2024-06-30 23:59:59"
end_test = "2024-07-31 23:59:59"

start_train_date = "2024-01-01"
start_val_date = "2024-06-17"
start_test_date = "2024-07-01"
end_train_date = "2024-06-16"
end_val_date = "2024-06-30"
end_test_date = "2024-07-31"

freq = "h"

In [4]:
threshold = 6

def remove_first_and_last_days(raw_data):
    raw_data = raw_data.copy()
    first_day = raw_data.index[0].strftime("%Y-%m-%d")
    last_day = raw_data.index[-1].strftime("%Y-%m-%d")
    
    raw_data = raw_data.drop(raw_data.loc[first_day].index)
    raw_data = raw_data.drop(raw_data.loc[last_day].index)
    
    return raw_data

def remove_last_day(raw_data):
    raw_data = raw_data.copy()
    # first_day = raw_data.index[0].strftime("%Y-%m-%d")
    last_day = raw_data.index[-1].strftime("%Y-%m-%d")
    
    # raw_data = raw_data.drop(raw_data.loc[first_day].index)
    raw_data = raw_data.drop(raw_data.loc[last_day].index)
    
    return raw_data

def add_extra_hour_in_beginning(raw_data):
    raw_data = raw_data.copy()
    first_index = raw_data.index[0]
    hour_before_first_index = first_index - pd.Timedelta(hours=1)
    hour_before_first_index = pd.Timestamp(hour_before_first_index)
    raw_data.loc[hour_before_first_index] = raw_data.loc[first_index]
    raw_data = raw_data.sort_index()
    
    return raw_data

def remove_incomplete_days(raw_data):
    raw_data = raw_data.copy()
    grouped_raw_data = raw_data.groupby(by=raw_data.index.date).count()
    incomplete_days = grouped_raw_data[grouped_raw_data != 24].index
    raw_data = remove_dates_from_raw_data(raw_data, incomplete_days, "Removing Incomplete Days")
    
    return raw_data

# This function assumes that the incoming raw data has a datetime index
def preprocess_raw_data(data, column_name):
    data = data.copy()
    data = add_extra_hour_in_beginning(data)
    data = data.resample(rule='h', closed='left', label='right').mean()
    data = remove_last_day(data)
    data = data[column_name]
    filtered_data, nan_values_filtered = remove_nan_days_above_threshold(threshold, data)
    filtered_data = filtered_data.interpolate(method='time')
    filtered_data = remove_incomplete_days(filtered_data)
    
    
    return filtered_data

def remove_dates_from_raw_data(raw_data, dates, comment):
    print(f"{comment} - Total Dropped Days: ", len(dates))    
    filtered_data = raw_data.copy()

    for date in dates:
        date = date.strftime("%Y-%m-%d")
        filtered_data = filtered_data.drop(filtered_data.loc[date].index)

    return filtered_data

def remove_nan_days_above_threshold(threshold, raw_data):
    nan_values = raw_data[raw_data.isna()].fillna(0)
    nan_values = nan_values.groupby(by=nan_values.index.date).count()
    nan_values.index = pd.to_datetime(nan_values.index)
    nan_values_filtered = nan_values[nan_values > threshold]
    filtered_data = remove_dates_from_raw_data(raw_data, nan_values_filtered.index, "Removing Nan Values")
    

    return filtered_data, nan_values_filtered

def remove_space_in_column_names(data):
    data.columns = data.columns.str.replace(' ', '_')
    return data

def get_raw_data_from_path(filepath):
    raw_data = pd.read_csv(
        filepath,
        na_values=[""],
        skipinitialspace=True,
        index_col='Time',
        parse_dates=True,
    ).dropna()
    
    raw_data.index = pd.to_datetime(raw_data.index).tz_localize(None)
    raw_data = raw_data[~raw_data.index.duplicated(keep='last')]

    return raw_data

def minmax_scale(data, min, max):
    return (data - min) / (max - min)

def minmax_inverse_scale(data, min, max):
    return (data * (max - min)) + min


In [5]:
holiday_years=[2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]

def get_barbados_holidays_list(years):
    barbados_holidays_list = holidays.Barbados(years=years)
    barbados_holidays_df = pd.DataFrame(barbados_holidays_list.values(), index=pd.to_datetime(list(barbados_holidays_list.keys())))
    return barbados_holidays_df

def get_unique_dates_from_hourly_data(data):
    return pd.to_datetime(data.index.date).unique()

def get_one_hot_day_of_week(dates_index):
    day_of_week = pd.get_dummies(dates_index.dayofweek).astype(int)
    day_of_week.index = dates_index
    day_of_week.columns = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return day_of_week

def add_is_holiday_column_to_one_hot_day_of_week(day_of_week, barbados_holidays_df):
    day_of_week['is_holiday'] = day_of_week.index.isin(barbados_holidays_df.index).astype(int)
    return day_of_week

barbados_holidays_df = get_barbados_holidays_list(holiday_years)
barbados_holidays_df


,0
2020-01-01,New Year's Day
2020-01-21,Errol Barrow Day
2020-04-10,Good Friday
2020-04-13,Easter Monday
2020-04-28,National Heroes Day
...,...
2030-08-01,Emancipation Day
2030-08-05,Kadooment Day
2030-11-30,Independence Day
2030-12-25,Christmas Day


In [6]:
hours_in_day = 15
target_column = 'NetLoadDemand'

def reshape_X_to_LSTM_shape(X):
    return X.reshape(X.shape[0], 1, X.shape[1])

def create_X_y(data, target_column, hours_in_day, day_of_week):
    X = []
    y_bl = []
    y_cl = []
    
    prev_dates = []
    cur_dates = []
    
    for i in range(hours_in_day, len(data), hours_in_day):
        cur = i
        prev = i - hours_in_day
        
        if(data.index[cur] != data.index[prev] + pd.Timedelta(days=1)):
            print(f'ALERT: Current: {data.index[cur]} to {data.index[cur + hours_in_day - 1]}, Previous: {data.index[prev]} to {data.index[prev + hours_in_day - 1]}')
            print('ALERT: Day Change Detected')
            continue
        
        print(f'Current: {data.index[cur]} to {data.index[cur + hours_in_day - 1]}, Previous: {data.index[prev]} to {data.index[prev + hours_in_day - 1]}')
        
        prev_historic_temp = data['temperature_2m_historic'].iloc[prev:cur].values.flatten()
        prev_forecasted_temp = data['temperature_2m_forecast'][prev:cur].values.flatten()
        prev_historic_shortwave_radiation = data['shortwave_radiation_historic'].iloc[prev:cur].values.flatten()
        prev_forecasted_shortwave_radiation = data['shortwave_radiation_forecast'][prev:cur].values.flatten()
        prev_load_demand = data[target_column].iloc[prev:cur].values.flatten()
        
        cur_historic_temp = data['temperature_2m_historic'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_forecasted_temp = data['temperature_2m_forecast'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_historic_shortwave_radiation = data['shortwave_radiation_historic'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_forecasted_shortwave_radiation = data['shortwave_radiation_forecast'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_load_demand = data[target_column].iloc[cur:cur + hours_in_day].values.flatten()
        cur_day_of_week = day_of_week.iloc[cur//hours_in_day].values.flatten()
        
        X_day = np.hstack([prev_load_demand, prev_historic_temp, prev_forecasted_temp, cur_historic_temp, cur_forecasted_temp, prev_historic_shortwave_radiation, prev_forecasted_shortwave_radiation, cur_historic_shortwave_radiation, cur_forecasted_shortwave_radiation, cur_day_of_week])
        y_day_bl = cur_load_demand
        y_day_cl = cur_load_demand - prev_load_demand
        
        X.append(X_day)
        y_bl.append(y_day_bl)
        y_cl.append(y_day_cl)
        
        prev_day_date = data.index[prev:cur].values.flatten()
        cur_day_date = data.index[cur:cur + hours_in_day].values.flatten()
        
        prev_dates.append(prev_day_date)
        cur_dates.append(cur_day_date)
    
    
    X = np.array(X)
    X_LSTM = reshape_X_to_LSTM_shape(X)
    y_bl = np.array(y_bl)
    y_cl = np.array(y_cl)
    
    cur_dates = np.array(cur_dates)
    prev_dates = np.array(prev_dates)
    
    return X, X_LSTM, y_bl, y_cl, cur_dates, prev_dates

def convert_cl_to_bl(y_pred_cl, y, X):
    y_prev_bl = X[:, :hours_in_day]
    
    if(y_prev_bl.shape == y_pred_cl.shape):
        y_pred_cl_bl = y_pred_cl + y_prev_bl
    
    return y_pred_cl_bl


In [7]:
# callbacks=[EarlyStopping(patience=50), ReduceLROnPlateau(patience=20, factor=0.1, min_lr=0.000000001)]
callbacks = None

def train_ann_model(X_train, y_train, callbacks=callbacks):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(46, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[MeanAbsoluteError(), RootMeanSquaredError()])
    model.summary()

    model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=callbacks)
    
    return model

def train_lstm_model(X_train_LSTM, y_train, callbacks=callbacks):
    model = Sequential()
    model.add(Input(shape=(X_train_LSTM.shape[1], X_train_LSTM.shape[2])))
    model.add(LSTM(150, return_sequences=False))
    model.add(Dense(y_train.shape[1], activation='linear'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[MeanAbsoluteError(), RootMeanSquaredError()])
    model.summary()

    model.fit(X_train_LSTM, y_train, epochs=100, batch_size=32, callbacks=callbacks)
    
    return model

def save_trained_model(model, path, model_type, feeder_filesave_name):
    model_save_path = f'{path}/{model_type}/{feeder_filesave_name}.keras'
    model.save(model_save_path)
    
    return

In [8]:
feeders_metadata = pd.read_csv(feeders_metadata_path)
weather_utils = WeatherUtils()
train_data_stats = []

for i in range(feeders_metadata.shape[0]):
    feeder_metadata = feeders_metadata.iloc[i]
    feeder_name = feeder_metadata['FeederName']
    feeder_data_filename = feeder_metadata['FileName']
    feeder_filesave_name = feeder_metadata['FileSaveName']
    feeder_data_path = feeders_root_path + feeder_data_filename
    ann_save_path = f"{saved_models_path}/ANN_Saved_Models/{feeder_filesave_name}_ANN_Model.keras"
    lstm_save_path = f"{saved_models_path}/LSTM_Saved_Models/{feeder_filesave_name}_LSTM_Model.keras"
    rls_combiner_save_path = f"{saved_models_path}/RLS_Combiner_Saved_Models/{feeder_filesave_name}_RLS_Combiner.pkl"
    feeder_data = get_raw_data_from_path(feeder_data_path)
    net_load_demand = preprocess_raw_data(feeder_data, "NetLoadDemand")
    latitude = feeder_metadata['Latitude']
    longitude = feeder_metadata['Longitude']
    
    
    print("========================================")
    print(feeder_name)
    print("========================================")
    
    historic_weather_data = weather_utils.fetch_historic_data_from_api(latitude, longitude, start_train_date, end_train_date)
    forecast_weather_data = weather_utils.fetch_forecast_data_from_api(latitude, longitude, start_train_date, end_train_date)
    
    temperature_historic = historic_weather_data["temperature_2m_historic"]
    temperature_forecast = forecast_weather_data["temperature_2m_forecast"]
    shortwave_radiation_historic = historic_weather_data["shortwave_radiation_historic"]
    shortwave_radiation_forecast = forecast_weather_data["shortwave_radiation_forecast"]
    
    
    combined_data = pd.concat([net_load_demand, temperature_historic, temperature_forecast, shortwave_radiation_historic, shortwave_radiation_forecast], axis=1).dropna()
    combined_data = combined_data.loc[:end_train]
    combined_data_daytime = combined_data.between_time('06:00', '20:00')
    
    mean = combined_data_daytime.mean()
    std = combined_data_daytime.std()
    min = combined_data_daytime.min()
    max = combined_data_daytime.max()
    
    train_stats = pd.concat([mean, std, min, max], axis=1)
    train_stats.columns = ['Mean', 'Std', 'Min', 'Max']
    
    normalized_data = minmax_scale(combined_data_daytime, train_stats['Min'], train_stats['Max'])
    print(normalized_data.describe())
    
    denormalized_data = minmax_inverse_scale(normalized_data, train_stats['Min'], train_stats['Max'])
    print(denormalized_data.describe())
    
    train_stats.to_csv(f"{train_stats_path}/{feeder_filesave_name}_train_stats.csv")
    
    dates_index = get_unique_dates_from_hourly_data(normalized_data)
    day_of_week = get_one_hot_day_of_week(dates_index)
    day_of_week = add_is_holiday_column_to_one_hot_day_of_week(day_of_week, barbados_holidays_df)
    
    X_train, X_train_LSTM, y_train_bl, y_train_cl, cur_dates_train, prev_dates_train = create_X_y(normalized_data, target_column, hours_in_day, day_of_week)
    
    trained_ann_model_bl = train_ann_model(X_train, y_train_bl)
    trained_ann_model_cl = train_ann_model(X_train, y_train_cl)
    trained_lstm_model_bl = train_lstm_model(X_train_LSTM, y_train_bl)
    trained_lstm_model_cl = train_lstm_model(X_train_LSTM, y_train_cl)
    
    save_trained_model(trained_ann_model_bl, saved_models_path, "ANN_BL", feeder_filesave_name)
    save_trained_model(trained_ann_model_cl, saved_models_path, "ANN_CL", feeder_filesave_name)
    save_trained_model(trained_lstm_model_bl, saved_models_path, "LSTM_BL", feeder_filesave_name)
    save_trained_model(trained_lstm_model_cl, saved_models_path, "LSTM_CL", feeder_filesave_name)
    
    print(X_train.shape, X_train_LSTM.shape, y_train_bl.shape, y_train_cl.shape, cur_dates_train.shape, prev_dates_train.shape)

# feeder_stats.to_csv(feeder_stats_path, index=False)

# arch_hall_feeder_data_path = feeders_root_path + "Arch_Hall_ST2B13.csv"
# arch_hall_feeder_data = get_raw_data_from_path(arch_hall_feeder_data_path)
# arch_hall_feeder_data = add_extra_hour_in_beginning(arch_hall_feeder_data)
# arch_hall_net_load_demand = preprocess_raw_data(arch_hall_feeder_data, "NetLoadDemand")
# arch_hall_net_load_demand

Removing Nan Values - Total Dropped Days:  3
Removing Incomplete Days - Total Dropped Days:  0
Arch Hall
Coordinates 13.181018829345703°N -59.56243896484375°E
Elevation 114.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
Coordinates 13.125°N -59.625°E
Elevation 114.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
       NetLoadDemand  temperature_2m_historic  temperature_2m_forecast  shortwave_radiation_historic  shortwave_radiation_forecast
count        2490.00                  2490.00                  2490.00                       2490.00                       2490.00
mean            0.49                     0.54                     0.50                          0.39                          0.41
std             0.20                     0.19                     0.20                          0.31                          0.33
min             0.00                     0.00                     0.00                    